In [4]:
import pandas as pd 

df = pd.read_csv('tvae_synthetic_dataset_cleaned.csv')
df.head()

,Unnamed: 0,PO_NUM,ITEM_CODE,ITEM_NAME,SUPPLIER_NAME,SUPPLIER_CODE,ORDERED_QUANTITY,PRICE,PO_VALUE,DOWNPAYMENT_DATE,DELIVERY_DATE,PART_DESCRIPTION
0,0,53586,14873,wire,SECURITAS ENGINEERS,VD000471,275.9,209.326401,0.0000,2/27/2023,6/10/2023,"['power', 'control', 'cable', ';', '3g4', ';',..."
1,1,233001368,2744400,ELCB,SECURITAS ENGINEERS,VD000471,57.8,3886.681121,136381.0189,2/27/2023,11/30/2023,"['earth', 'leakage', 'circuit', 'breaker', 'dp..."
2,2,977417,2277700,Ethernet I/O module,TAC Automation Pvt.Ltd- UP,VD002456,16.2,10244.522240,249007.2534,2/27/2023,2/2/2023,"['compact', 'ip20', 'multiprotocol', 'ethernet..."
3,3,53586,18054,Cable,IGUS INDIA PRIVATE LIMITED,VD000478,50.0,1300.563242,401538.8736,2/27/2023,3/30/2023,"['db25', 'male', 'breakout', 'cable', 'db9', '..."
4,4,233033749,2656800,BUSBAR ALUMINUM,MEERA ENTERPRISES,VD003901,9.7,0.010000,0.0000,2/27/2023,2/10/2024,"['busbar', '25x6mm', 'aluminum', '25', 'mtr', ..."


In [5]:
df.columns

Index(['Unnamed: 0', 'PO_NUM', 'ITEM_CODE', 'ITEM_NAME', 'SUPPLIER_NAME',
       'SUPPLIER_CODE', 'ORDERED_QUANTITY', 'PRICE', 'PO_VALUE',
       'DOWNPAYMENT_DATE', 'DELIVERY_DATE', 'PART_DESCRIPTION'],
      dtype='object')

In [6]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [8]:
# cast columns to correct data types
df["PART_DESCRIPTION"] = df["PART_DESCRIPTION"].to_list()

In [13]:
# create a df with unique item code supplier code pairs their corresponding part descriptions
df_unique_items = df.groupby(['ITEM_CODE', 'SUPPLIER_CODE'])['PART_DESCRIPTION'].apply(list).reset_index()
print(df_unique_items.shape)
df_unique_items.head(10)

(7166, 3)


,ITEM_CODE,SUPPLIER_CODE,PART_DESCRIPTION
0,13271,VD000692,"[['somanet', 'circulo', '9', 'io', 'cable', 'k..."
1,13271,VD001513,"[['somanet', 'circulo', '9', 'io', 'cable', 'k..."
2,14730,VD000532,"[['single', 'board', 'computer', 'intel', 'n42..."
3,14730,VD001614,"[['48vdc', 'brushless', 'servo', 'motor', '150..."
4,14730,VD001966,"[['single', 'board', 'computer', 'intel', 'n42..."
5,14730,VD002495,"[['busbar', 'ecu', 'length', ':', '2400', 'mmb..."
6,14858,VD001275,"[['somanet', 'circulo', '9', 'io', 'cable', 'k..."
7,14869,VD000471,"[['power', 'control', 'cable', ';', '3g4', ';'..."
8,14869,VD001209,"[['025', 'sqmm', 'pin', 'type', 'lug']]"
9,14870,VD000471,"[['power', 'control', 'cable', ';', '3g4', ';'..."


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
import numpy as np

# create a count vectorizer
count = CountVectorizer()
count_matrix = count.fit_transform(df_unique_items['PART_DESCRIPTION'])

# create a jaccard similarity matrix
jaccard_sim = np.zeros((len(df), len(df)))
for i in range(len(df)):
    for j in range(len(df)):
        tokens_i = count.transform([df['PART_DESCRIPTION'][i]]).toarray()[0]
        tokens_j = count.transform([df['PART_DESCRIPTION'][j]]).toarray()[0]
        jaccard_sim[i, j] = jaccard_score(tokens_i, tokens_j)

rows = []

for i in range(len(df)):
    for j in range(len(df)):
        row = {
            'ITEM_CODE_i': df.loc[i, 'ITEM_CODE'],
            'SUPPLIER_CODE_i': df.loc[i, 'SUPPLIER_CODE'],
            'ITEM_CODE_j': df.loc[j, 'ITEM_CODE'],
            'SUPPLIER_CODE_j': df.loc[j, 'SUPPLIER_CODE'],
            'Jaccard_Similarity': jaccard_sim[i, j]
        }
        rows.append(row)

df_jaccard = pd.DataFrame(rows, columns=['ITEM_CODE_i', 'SUPPLIER_CODE_i', 'ITEM_CODE_j', 'SUPPLIER_CODE_j', 'Jaccard_Similarity'])
df_jaccard.head()

# jaccard_sim now contains the Jaccard similarity matrix

AttributeError: 'list' object has no attribute 'lower'